## Ch15 Ex1 Email DB

In [26]:
import sqlite3

In [27]:
conn = sqlite3.connect('emaildb.sqlite')
cur = conn.cursor()
# con : 특정 db 파일과 connect 하는 것
# 현재는 emaildb.sqlite 파일이 없으므로 새로 생성하게 된다.
# cursor 메소드는 위의 연결을 사용하겠다는 것으로 보인다.

In [28]:
cur.execute('''DROP TABLE IF EXISTS Counts''');
# 만약 emaildb 라는 곳에 Counts 라는 테이블이 남아있다면, 삭제한다.

In [29]:
cur.execute('''CREATE TABLE Counts (email TEXT, count INTEGER)''');
# Counts 테이블을 생성하여 email, count 컬럼을 설정한다.

In [30]:
fname = input('Enter file name: ')
if(len(fname) < 1):
    fname = 'mbox-short.txt'
fh = open(fname)
# 특정 파일의 이름을 입력하여, 해당 파일을 읽을 수 있도록 open 한다.
# 만약 아무 이름을 입력하지 않았다면, mbox-short 라는 텍스트 파일을 open 한다

Enter file name: 


In [31]:
for line in fh:
    if not line.startswith('From: '):
        continue
        
    # file 을 열어 한 줄씩 읽어오며, From 으로 시작하는지를 검사한다.
    # 만약 시작하지 않는다면 다음 줄로 넘어가 계속 진행한다.
    
    pieces = line.split()
    email = pieces[1]
    cur.execute('''SELECT count FROM Counts WHERE email = ?''', (email,))
    row = cur.fetchone()

    # 만약 From 으로 시작하는 문장을 찾았다면, 띄어쓰기로 나누어 pieces에 리스트 형태로 저장한다.
    # email 은 pieces[0]인 From: 다음의 pieces[1] 이므로 해당 부분을 email 변수에 저장한다.
    # emaildb 에서 방금 뽑아낸 pieces[1] 이메일값을 email 로 가지고 있는 행을 검색한다.
    # execute 는 쿼리문을 실행시키는 명령어이며, 이 명령어의 결과값을 뽑아내기 위해서는 fetchall / fetchone 을 사용한다.
    # 하나의 행만이 결과이겠지만, 어쨌든 fetchone 으로 하나의 행을 뽑아내어 row 에 저장한다.
    
    if row is None:
        cur.execute('''INSERT INTO Counts (email, count) VALUES (?, 1)''',
                   (email,))
    else:
        cur.execute('''UPDATE Counts SET count = count + 1 WHERE email = ?''',
                   (email,))
    conn.commit()
    
    # 만약 row 가 존재하지 않는다면, 해당 email 에 대해 count 가 1이 되도록 행을 추가한다.
    # 만약 row 가 존재한다면, 해당 email 에 대해 count 가 기존의 값에서 1 증가하도록 행을 수정한다.
    # 최종적으로 commit 메소드를 이용해 sql 에 명령어를 적용시키면 sql 에 값이 쓰여진다.

In [32]:
sqlstr = 'SELECT email, count FROM Counts ORDER BY count DESC LIMIT 5'
# Counts 테이블에서 count 컬럼을 내림차순으로 정렬하여 가장 위의 5개의 행에 대해 
# email과 count 값을 뽑아내는 쿼리문을 sqlstr 에 저장한다.

In [33]:
for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])
    # 위의 쿼리문에 대해 execute 로 뽑아낸 값을 row 단위로 나누면 email 과 count 를 가진 리스트가 반환되며
    # 해당 리스트의 0번째 값과 1번째 값을 str 형태로 순차적으로 출력하면 email 과 count 가 출력된다.

cwen@iupui.edu 5
zqian@umich.edu 4
david.horwitz@uct.ac.za 4
louis@media.berkeley.edu 3
gsilver@umich.edu 3


In [34]:
cur.close()
# 마지막으로 sqlite 의 connection 을 close 한다.